In [ ]:
import torch 
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from torch import device, cuda
from torch.optim import Adam
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, ConfusionMatrixDisplay
import numpy as np 
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from time import time
import os

import sys
sys.path.append('..')  
from src.utils.RawDataReader import RawDataReader


In [ ]:
reader = RawDataReader()
labels = reader.get_labels()
datasets, means, stds = reader.get_normalised_dataset()

In [ ]:
from torch.nn import Module, Sequential, Conv1d, LeakyReLU, MaxPool1d, Flatten, Linear, Sigmoid

class SingleHeadCNNModel(Module):
    def __init__(self, feature_count:int, hidden_count:int=5):
        super(SingleHeadCNNModel, self).__init__()

        self.slope = 0.001

        self.channel_counts = 2

        self.cnn_model = Sequential(
            *[self.cnn_block((self.channel_counts**_h_index),(self.channel_counts**(_h_index+1))) for _h_index in range(hidden_count)],
            Flatten()
        )
        self.fc_model = Sequential(
            self.linear_block(in_features=feature_count, out_features=100),
            self.linear_block(in_features=100, out_features=10),
            self.linear_block(in_features=10, out_features=1),
            Sigmoid()
        )

    def cnn_block(self, in_channels:int, out_channels:int, kernel_size:int=3, padding=1):
        return Sequential(
            Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=padding),
            LeakyReLU(negative_slope=self.slope),
            MaxPool1d(kernel_size=2)
        )
    
    def linear_block(self, in_features, out_features):
        return Sequential(
            Linear(in_features=in_features, out_features=out_features),
            LeakyReLU(negative_slope=self.slope)
        )
    
    def forward(self, x):
        return self.fc_model(self.cnn_model(x))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cuda


In [13]:
r = 2 
batch_size = 2048
learning_rate = 0.01
test_split = 0.2
epochs = 9999

random_state = 1
hidden_count = 0

num_lables = labels.shape[1]

In [14]:
curr_dataset = datasets[r]
feature_count = curr_dataset.shape[1]
for label_index in range(num_lables):
    curr_label = labels[:, [label_index]]
    x_train, x_test, y_train, y_test = train_test_split(curr_dataset, curr_label, test_size=test_split, random_state=random_state, stratify=curr_label)
    x_train = x_train.unsqueeze(1).to(device)
    y_train.to(device)
    x_test = x_test.unsqueeze(1).to(device)

    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    head_model = SingleHeadCNNModel(feature_count=feature_count, hidden_count=hidden_count)
    head_model.to(device)
    
    criterion = torch.nn.BCELoss().to(device)
    optimizer = Adam(head_model.parameters(), lr=learning_rate)

    for epoch in tqdm(range(epochs)):
        head_model.train()
        total_loss = 0
        for batch_features, batch_labels in train_loader:
            optimizer.zero_grad()
            predictions = head_model(batch_features.to(device))
            loss = criterion(predictions, batch_labels.to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss
        total_loss = (total_loss / len(train_loader)).item()

        y_test_pred = head_model(x_test)
        f1 = f1_score(y_test,(y_test_pred.cpu() >0.5).float())
        print(total_loss, f1)
    break
    

  0%|          | 2/9999 [00:00<16:59,  9.80it/s]

0.6787839531898499 0.5960784313725491
0.6609547734260559 0.6406685236768802


  0%|          | 4/9999 [00:00<20:20,  8.19it/s]

0.6508848667144775 0.6505576208178439
0.6331400275230408 0.6580406654343808


  0%|          | 6/9999 [00:00<21:35,  7.71it/s]

...

  3%|▎         | 260/9999 [00:33<20:51,  7.78it/s]

0.08172670751810074 0.715
0.09128759056329727 0.6969178082191781
0.08331949263811111 0.7222222222222222


  3%|▎         | 262/9999 [00:33<20:52,  7.78it/s]

0.08311136066913605 0.7140468227424749


KeyboardInterrupt: 

In [ ]:
head_model(curr_dataset.unsqueeze(1).to(device))

tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]], device='cuda:0', grad_fn=<SigmoidBackward0>)